In [1]:
import theano
import numpy as np
import sys
import pandas as pd
import scipy
from scipy.stats import spearmanr

%matplotlib inline
import matplotlib.pyplot as plt

We will walk through the basic functions of loading up a model and predicting the effects of mutations.

# Downloading pretrained parameters

Please first download the pretrained parameters with download_pretrained.sh. 

# Loading the model

In [2]:
sys.path.insert(0, "../DeepSequence")

import model
import helper
import train

# Mutation effect prediction

Mutation effect prediction helper functions are always with respect to the focus sequence of the alignment. We can ask for a prediction of mutation effect individually.

For reliable mutation effect prediction results, we recommend taking Monte Carlo 500-2000 samples from the model (with the N_pred_iterations parameter).

We can predict the effects of single, double, triple mutants, etc. Mutations are organized as a list of tuples, where the tuples are (uniprot position, wt amino acid, mutant amino acid).

# PABP

First let's load up a model. We don't have to calculate sequence weights here because we are not training a model and this can be slow on the CPU. 

In the "Explore model parameters.ipynb" notebook, the helper.py code was ammended to prespesify a dataset used for the DataHelper class. However, we can pass in an alignment name and a few more parameters so we don't have to modify the helper.py file.

In [3]:
data_params = {"alignment_file":"datasets/PABP_YEAST_hmmerbit_plmc_n5_m30_f50_t0.2_r115-210_id100_b48.a2m"}

pabp_data_helper = helper.DataHelper(
                alignment_file=data_params["alignment_file"],
                working_dir=".",
                calc_weights=False
                )

model_params = {
        "batch_size"        :   100,
        "encode_dim_zero"   :   1500,
        "encode_dim_one"    :   1500,
        "decode_dim_zero"   :   100,
        "decode_dim_one"    :   500,
        "n_patterns"        :   4,
        "n_latent"          :   30,
        "logit_p"           :   0.001,
        "sparsity"          :   "logit",
        "encode_nonlin"     :   "relu",
        "decode_nonlin"     :   "relu",
        "final_decode_nonlin":  "sigmoid",
        "output_bias"       :   True,
        "final_pwm_scale"   :   True,
        "conv_pat"          :   True,
        "d_c_size"          :   40
        }

pabp_vae_model   = model.VariationalAutoencoder(pabp_data_helper,
    batch_size              =   model_params["batch_size"],
    encoder_architecture    =   [model_params["encode_dim_zero"],
                                model_params["encode_dim_one"]],
    decoder_architecture    =   [model_params["decode_dim_zero"],
                                model_params["decode_dim_one"]],
    n_latent                =   model_params["n_latent"],
    n_patterns              =   model_params["n_patterns"],
    convolve_patterns       =   model_params["conv_pat"],
    conv_decoder_size       =   model_params["d_c_size"],
    logit_p                 =   model_params["logit_p"],
    sparsity                =   model_params["sparsity"],
    encode_nonlinearity_type       =   model_params["encode_nonlin"],
    decode_nonlinearity_type       =   model_params["decode_nonlin"],
    final_decode_nonlinearity      =   model_params["final_decode_nonlin"],
    output_bias             =   model_params["output_bias"],
    final_pwm_scale         =   model_params["final_pwm_scale"],
    working_dir             =   ".")

print ("Model built")

Encoding sequences
Neff = 151528.0
Data Shape = (151528, 82, 20)
Model built


Load up the parameters of a pretrained model in the 'params' folder.

In [4]:
file_prefix = "PABP_YEAST"
pabp_vae_model.load_parameters(file_prefix=file_prefix)
print ("Parameters loaded")

Parameters loaded


In [6]:
print (pabp_data_helper.delta_elbo(pabp_vae_model,[(126,"G","A")], N_pred_iterations=500))

-2.03463650668


In [7]:
print (pabp_data_helper.delta_elbo(pabp_vae_model,[(126,"G","A"), (137,"I","P")], N_pred_iterations=500))

-10.8308351474


In [8]:
print (pabp_data_helper.delta_elbo(pabp_vae_model,[(126,"G","A"), (137,"I","P"), (155,"S","A")], N_pred_iterations=500))

-16.058655309


We can predict the effects of mutations for all single mutations. This and the below function are preferred because they can take advantages of speed-ups from minibatching the mutation data.

In [9]:
pabp_full_matr_mutant_name_list, pabp_full_matr_delta_elbos \
    = pabp_data_helper.single_mutant_matrix(pabp_vae_model, N_pred_iterations=500)

In [10]:
print (pabp_full_matr_mutant_name_list[0], pabp_full_matr_delta_elbos[0])

('K123A', 0.5887526915685584)


We can also predict the effect of mutations from a file in batched mode.

In [11]:
pabp_custom_matr_mutant_name_list, pabp_custom_matr_delta_elbos \
    = pabp_data_helper.custom_mutant_matrix("mutations/PABP_YEAST_Fields2013-singles.csv", \
                                            pabp_vae_model, N_pred_iterations=500)
    
print (pabp_custom_matr_mutant_name_list[12], pabp_custom_matr_delta_elbos[12])

('N127D', -6.426795215037501)


Let's also make a quick function to calculate the spearman rho from a mutation file.

In [12]:
def generate_spearmanr(mutant_name_list, delta_elbo_list, mutation_filename, phenotype_name):
    
    measurement_df = pd.read_csv(mutation_filename, sep=',')

    mutant_list = measurement_df.mutant.tolist()
    expr_values_ref_list = measurement_df[phenotype_name].tolist()

    mutant_name_to_pred = {mutant_name_list[i]:delta_elbo_list[i] for i in range(len(delta_elbo_list))}
    
    # If there are measurements 
    wt_list = []
    preds_for_spearmanr = []
    measurements_for_spearmanr = []
    
    for i,mutant_name in enumerate(mutant_list):
        expr_val = expr_values_ref_list[i]
        
        # Make sure we have made a prediction for that mutant
        if mutant_name in mutant_name_to_pred:
            multi_mut_name_list = mutant_name.split(':')
        
            # If there is no measurement for that mutant, pass over it
            if np.isnan(expr_val):
                pass

            # If it was a codon change, add it to the wt vals to average
            elif mutant_name[0] == mutant_name[-1] and len(multi_mut_name_list) == 1:
                wt_list.append(expr_values_ref_list[i])

            # If it is labeled as the wt sequence, add it to the average list
            elif mutant_name == 'wt' or mutant_name == 'WT':
                wt_list.append(expr_values_ref_list[i])

            else:
                measurements_for_spearmanr.append(expr_val)
                preds_for_spearmanr.append(mutant_name_to_pred[mutant_name])

    if wt_list != []:
        measurements_for_spearmanr.append(np.mean(average_wt_list))
        preds_for_spearmanr.append(0.0)

    num_data = len(measurements_for_spearmanr)
    spearman_r, spearman_pval = spearmanr(measurements_for_spearmanr, preds_for_spearmanr)
    print ("N: "+str(num_data)+", Spearmanr: "+str(spearman_r)+", p-val: "+str(spearman_pval))

In [13]:
generate_spearmanr(pabp_custom_matr_mutant_name_list, pabp_custom_matr_delta_elbos, \
                   "mutations/PABP_YEAST_Fields2013-singles.csv", "log")

N: 1188, Spearmanr: 0.6509305755221257, p-val: 4.0800344026520655e-144


# PDZ

In [19]:
data_params = {"alignment_file":"datasets/DLG4_RAT_hmmerbit_plmc_n5_m30_f50_t0.2_r300-400_id100_b50.a2m"}

pdz_data_helper = helper.DataHelper(
                alignment_file=data_params["alignment_file"],
                working_dir=".",
                calc_weights=False
                )

pdz_vae_model   = model.VariationalAutoencoder(pdz_data_helper,
    batch_size              =   model_params["batch_size"],
    encoder_architecture    =   [model_params["encode_dim_zero"],
                                model_params["encode_dim_one"]],
    decoder_architecture    =   [model_params["decode_dim_zero"],
                                model_params["decode_dim_one"]],
    n_latent                =   model_params["n_latent"],
    n_patterns              =   model_params["n_patterns"],
    convolve_patterns       =   model_params["conv_pat"],
    conv_decoder_size       =   model_params["d_c_size"],
    logit_p                 =   model_params["logit_p"],
    sparsity                =   model_params["sparsity"],
    encode_nonlinearity_type       =   model_params["encode_nonlin"],
    decode_nonlinearity_type       =   model_params["decode_nonlin"],
    final_decode_nonlinearity      =   model_params["final_decode_nonlin"],
    output_bias             =   model_params["output_bias"],
    final_pwm_scale         =   model_params["final_pwm_scale"],
    working_dir             =   ".")

print ("Model built")

file_prefix = "DLG4_RAT"
pdz_vae_model.load_parameters(file_prefix=file_prefix)

print ("Parameters loaded\n\n")

pdz_custom_matr_mutant_name_list, pdz_custom_matr_delta_elbos \
    = pdz_data_helper.custom_mutant_matrix("mutations/DLG4_RAT_Ranganathan2012.csv", \
                                            pdz_vae_model, N_pred_iterations=500)
  
generate_spearmanr(pdz_custom_matr_mutant_name_list, pdz_custom_matr_delta_elbos, \
                   "mutations/DLG4_RAT_Ranganathan2012.csv", "CRIPT")

Encoding sequences
Neff = 102246.0
Data Shape = (102246, 84, 20)
Model built
Parameters loaded


N: 1577, Spearmanr: 0.6199244929585085, p-val: 4.31636475994128e-168


# B-lactamase

Larger proteins with more mutations to predict can take much longer to run. For these, we recommend GPU-enabled computation.

In [27]:
data_params = {"dataset":"BLAT_ECOLX"}

blat_data_helper = helper.DataHelper(
                dataset=data_params["dataset"],
                working_dir=".",
                calc_weights=False
                )

blat_vae_model   = model.VariationalAutoencoder(blat_data_helper,
    batch_size              =   model_params["batch_size"],
    encoder_architecture    =   [model_params["encode_dim_zero"],
                                model_params["encode_dim_one"]],
    decoder_architecture    =   [model_params["decode_dim_zero"],
                                model_params["decode_dim_one"]],
    n_latent                =   model_params["n_latent"],
    n_patterns              =   model_params["n_patterns"],
    convolve_patterns       =   model_params["conv_pat"],
    conv_decoder_size       =   model_params["d_c_size"],
    logit_p                 =   model_params["logit_p"],
    sparsity                =   model_params["sparsity"],
    encode_nonlinearity_type       =   model_params["encode_nonlin"],
    decode_nonlinearity_type       =   model_params["decode_nonlin"],
    final_decode_nonlinearity      =   model_params["final_decode_nonlin"],
    output_bias             =   model_params["output_bias"],
    final_pwm_scale         =   model_params["final_pwm_scale"],
    working_dir             =   ".")

print ("Model built")

file_prefix = "BLAT_ECOLX"
blat_vae_model.load_parameters(file_prefix=file_prefix)

print ("Parameters loaded\n\n")

blat_custom_matr_mutant_name_list, blat_custom_matr_delta_elbos \
    = blat_data_helper.custom_mutant_matrix("mutations/BLAT_ECOLX_Ranganathan2015.csv", \
                                            blat_vae_model, N_pred_iterations=500)
    
generate_spearmanr(blat_custom_matr_mutant_name_list, blat_custom_matr_delta_elbos, \
                   "mutations/BLAT_ECOLX_Ranganathan2015.csv", "2500")

Encoding sequences
Neff = 8355.0
Data Shape = (8355, 253, 20)
Model built
Parameters loaded


N: 4807, Spearmanr: 0.743886370415797, p-val: 0.0
